# Pipeline Transaccional con Neo4j y LLM

Sistema de consulta transaccional sobre grafo de conocimiento usando:
- Neo4j: Base de datos de grafos
- Dataset real: Productos, vendedores, pedidos
- Embeddings: Para routing de funciones
- Google Gemini: Para respuestas en lenguaje natural

Pipeline: Query → Function Selection → Neo4j Execution → LLM Response

## 1. Instalación de Dependencias

In [1]:
# Instalación de paquetes necesarios
import sys
import subprocess


def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])


packages = [
    "neo4j",
    "pandas",
    "sentence-transformers",
    "python-dotenv",
    "scikit-learn",
    "numpy",
    "google-generativeai",
    "kagglehub",
]

for pkg in packages:
    try:
        __import__(pkg.replace("-", "_"))
        print(f"✓ {pkg} ya instalado")
    except ImportError:
        print(f"Instalando {pkg}...")
        install_package(pkg)

print("\nTodas las dependencias instaladas correctamente.")

✓ neo4j ya instalado
✓ pandas ya instalado


c:\Users\mlata\Documents\estocasticos\final\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


✓ sentence-transformers ya instalado
Instalando python-dotenv...
Instalando scikit-learn...
✓ numpy ya instalado
Instalando google-generativeai...
✓ kagglehub ya instalado

Todas las dependencias instaladas correctamente.


## 2. Imports y Configuración

In [2]:
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import google.generativeai as genai
import kagglehub
import os
from pathlib import Path
from dotenv import load_dotenv
import json
import sys

# Resolver rutas base
BASE_DIR = Path.cwd()
if not (BASE_DIR / "data").exists() and (BASE_DIR.parent / "data").exists():
    BASE_DIR = BASE_DIR.parent

MODEL_DIR = BASE_DIR / "model"
if MODEL_DIR.exists() and str(MODEL_DIR) not in sys.path:
    sys.path.append(str(MODEL_DIR))

# Cargar variables de entorno
load_dotenv(BASE_DIR / ".env")

# Configuración Neo4j (ajustar según tu instalación)
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password")

# Configuración Google (Gemini)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✓ Google Gemini API configurada")
else:
    print("⚠ WARNING: GOOGLE_API_KEY no configurada")

# Cargar modelo de embeddings
print("Cargando modelo de embeddings...")
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
print("✓ Modelo de embeddings cargado")

c:\Users\mlata\Documents\estocasticos\final\.venv\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✓ Google Gemini API configurada
Cargando modelo de embeddings...
✓ Modelo de embeddings cargado


## 3. Conexión a Neo4j

In [4]:
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def query(self, cypher_query, parameters=None):
        """Ejecuta una query Cypher y retorna resultados"""
        with self.driver.session() as session:
            result = session.run(cypher_query, parameters or {})
            return [record.data() for record in result]
    
    def execute(self, cypher_query, parameters=None):
        """Ejecuta una query Cypher sin retornar resultados"""
        with self.driver.session() as session:
            session.run(cypher_query, parameters or {})

# Crear conexión
try:
    neo4j_conn = Neo4jConnection(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    # Verificar conexión
    result = neo4j_conn.query("RETURN 1 as test")
    print("✓ Conexión a Neo4j exitosa")
except Exception as e:
    print(f"✗ Error conectando a Neo4j: {e}")
    print("\nInstrucciones:")
    print("1. Instala Neo4j Desktop: https://neo4j.com/download/")
    print("2. Crea una base de datos local")
    print("3. Configura las credenciales en .env o en el código")

✓ Conexión a Neo4j exitosa


## 4. Cargar Dataset

In [5]:
# Cargar dataset
kagglehub_dataset = "kartikeybartwal/ecomerce-product-recommendation-dataset"
DATASET_MODE = "local"


def _pick_column(columns, candidates):
    columns_lower = {c.lower(): c for c in columns}
    for c in candidates:
        if c in columns_lower:
            return columns_lower[c]
    for col in columns:
        for c in candidates:
            if c in col.lower():
                return col
    return None


# 1) Intentar KaggleHub (dataset recomendado)
try:
    kaggle_path = Path(kagglehub.dataset_download(kagglehub_dataset))
    csv_files = sorted(kaggle_path.rglob("*.csv"), key=lambda p: p.stat().st_size, reverse=True)
    if csv_files:
        raw_file = csv_files[0]
        df_raw = pd.read_csv(raw_file)
        print(f"Usando dataset KaggleHub: {kagglehub_dataset}")
        print(f"Archivo cargado: {raw_file.name}")

        # Detectar columnas
        product_id_col = _pick_column(df_raw.columns, ["product_id", "productid", "item_id", "itemid"])
        product_name_col = _pick_column(df_raw.columns, ["product_name", "product", "item_name", "name"])
        category_col = _pick_column(df_raw.columns, ["category", "product_category", "category_name"])
        price_col = _pick_column(df_raw.columns, ["price", "product_price", "amount"])
        user_col = _pick_column(df_raw.columns, ["user_id", "userid", "customer_id", "customerid"])
        rating_col = _pick_column(df_raw.columns, ["rating", "score", "review", "stars"])
        seller_col = _pick_column(df_raw.columns, ["seller_id", "seller", "vendor", "merchant"])

        if product_id_col is None:
            df_raw["_product_id"] = df_raw.index.astype(str)
            product_id_col = "_product_id"

        if product_name_col is None:
            product_name_col = product_id_col

        if category_col is None:
            df_raw["_categoria"] = "general"
            category_col = "_categoria"

        if price_col is None:
            df_raw["_precio"] = df_raw[rating_col] if rating_col else 0
            price_col = "_precio"

        if user_col is None:
            df_raw["_user_id"] = "anonimo"
            user_col = "_user_id"

        if seller_col is None:
            df_raw["_seller_id"] = "marketplace"
            seller_col = "_seller_id"

        df_productos = (
            df_raw[[product_id_col, product_name_col, category_col, price_col]]
            .drop_duplicates(subset=[product_id_col])
            .rename(
                columns={
                    product_id_col: "id",
                    product_name_col: "nombre",
                    category_col: "categoria",
                    price_col: "precio",
                }
            )
        )

        df_vendedores = (
            df_raw[[seller_col]]
            .drop_duplicates()
            .rename(columns={seller_col: "id"})
        )
        if "ciudad" not in df_vendedores.columns:
            df_vendedores["ciudad"] = "desconocido"
        if "estado" not in df_vendedores.columns:
            df_vendedores["estado"] = "desconocido"

        df_clientes = (
            df_raw[[user_col]]
            .drop_duplicates()
            .rename(columns={user_col: "id"})
        )
        if "ciudad" not in df_clientes.columns:
            df_clientes["ciudad"] = "desconocido"
        if "estado" not in df_clientes.columns:
            df_clientes["estado"] = "desconocido"

        df_interacciones = df_raw[[user_col, product_id_col]]
        if rating_col:
            df_interacciones = df_interacciones.assign(rating=df_raw[rating_col])
        else:
            df_interacciones = df_interacciones.assign(rating=1)

        df_interacciones = df_interacciones.rename(
            columns={user_col: "cliente_id", product_id_col: "producto_id"}
        )

        DATASET_MODE = "kagglehub"

        print(f"Productos cargados: {len(df_productos)}")
        display(df_productos.head())
        print(f"Vendedores cargados: {len(df_vendedores)}")
        display(df_vendedores.head())
        print(f"Clientes cargados: {len(df_clientes)}")
        display(df_clientes.head())
    else:
        print("No se encontraron CSV en el dataset KaggleHub.")
except Exception as e:
    print(f"KaggleHub no disponible o falló la descarga: {e}")

# 2) Intentar Olist si KaggleHub no se pudo usar
if DATASET_MODE == "local":
    olist_dir = BASE_DIR / "data" / "olist"
    olist_required = [
        "olist_orders_dataset.csv",
        "olist_order_items_dataset.csv",
        "olist_products_dataset.csv",
        "olist_sellers_dataset.csv",
        "olist_customers_dataset.csv",
    ]

    olist_available = all((olist_dir / f).exists() for f in olist_required)

    if olist_available:
        DATASET_MODE = "olist"
        print("Usando dataset Olist (Kaggle).")

        df_orders = pd.read_csv(olist_dir / "olist_orders_dataset.csv")
        df_order_items = pd.read_csv(olist_dir / "olist_order_items_dataset.csv")
        df_products_raw = pd.read_csv(olist_dir / "olist_products_dataset.csv")
        df_sellers = pd.read_csv(olist_dir / "olist_sellers_dataset.csv")
        df_customers = pd.read_csv(olist_dir / "olist_customers_dataset.csv")

        translation_file = olist_dir / "product_category_name_translation.csv"
        if translation_file.exists():
            df_translation = pd.read_csv(translation_file)
        else:
            df_translation = None

        # Agregar métricas de precio por producto
        item_stats = (
            df_order_items.groupby("product_id")
            .agg(
                precio_promedio=("price", "mean"),
                total_ordenes=("order_id", "nunique"),
                total_items=("order_item_id", "count"),
            )
            .reset_index()
        )

        df_productos = df_products_raw.merge(item_stats, on="product_id", how="left")

        if df_translation is not None:
            df_productos = df_productos.merge(
                df_translation,
                on="product_category_name",
                how="left",
            )
            df_productos["categoria"] = df_productos["product_category_name_english"].fillna(
                df_productos["product_category_name"]
            )
        else:
            df_productos["categoria"] = df_productos["product_category_name"]

        df_productos["nombre"] = df_productos["product_id"]

        df_vendedores = df_sellers.rename(
            columns={
                "seller_id": "id",
                "seller_city": "ciudad",
                "seller_state": "estado",
            }
        )

        df_clientes = df_customers.rename(
            columns={
                "customer_id": "id",
                "customer_city": "ciudad",
                "customer_state": "estado",
            }
        )

        print(f"Productos cargados: {len(df_productos)}")
        display(df_productos.head())
        print(f"Vendedores cargados: {len(df_vendedores)}")
        display(df_vendedores.head())
        print(f"Clientes cargados: {len(df_clientes)}")
        display(df_clientes.head())

# 3) Dataset local de ejemplo
if DATASET_MODE == "local":
    print("Usando dataset local de ejemplo.")

    productos_path = BASE_DIR / "data" / "dataset_transaccional.csv"
    vendedores_path = BASE_DIR / "data" / "dataset_vendedores.csv"

    df_productos = pd.read_csv(productos_path)
    print(f"Productos cargados: {len(df_productos)}")
    print("\nMuestra:")
    display(df_productos.head())

    df_vendedores = pd.read_csv(vendedores_path)
    print(f"\nVendedores cargados: {len(df_vendedores)}")
    display(df_vendedores)

Usando dataset KaggleHub: kartikeybartwal/ecomerce-product-recommendation-dataset
Archivo cargado: content_based_recommendation_dataset.csv
Productos cargados: 1474


,id,nombre,categoria,precio
0,0,12,general,500
1,1,8,general,3000
2,2,25,general,600
3,3,6,general,100
4,4,18,general,2000


Vendedores cargados: 1


,id,ciudad,estado
0,marketplace,desconocido,desconocido


Clientes cargados: 1


,id,ciudad,estado
0,anonimo,desconocido,desconocido


## 5. Cargar Datos a Neo4j

In [6]:
# Limpiar base de datos (cuidado en producción!)
print("Limpiando base de datos...")
neo4j_conn.execute("MATCH (n) DETACH DELETE n")


def batch_write(query, rows, batch_size=1000):
    for i in range(0, len(rows), batch_size):
        neo4j_conn.execute(query, {"rows": rows[i : i + batch_size]})


if DATASET_MODE == "kagglehub":
    print("\nCargando datos KaggleHub a Neo4j...")

    # Vendedores
    vendedores_rows = (
        df_vendedores[["id", "ciudad", "estado"]]
        .dropna(subset=["id"])
        .to_dict("records")
    )
    query_vendedores = """
    UNWIND $rows AS row
    MERGE (v:Vendedor {id: row.id})
    SET v.ciudad = row.ciudad,
        v.estado = row.estado
    """
    batch_write(query_vendedores, vendedores_rows)
    print(f"✓ {len(vendedores_rows)} vendedores cargados")

    # Productos
    productos_rows = (
        df_productos[["id", "nombre", "categoria", "precio"]]
        .fillna({"precio": 0, "categoria": "general"})
        .to_dict("records")
    )
    query_productos = """
    UNWIND $rows AS row
    MERGE (p:Producto {id: row.id})
    SET p.nombre = row.nombre,
        p.categoria = row.categoria,
        p.precio = row.precio
    """
    batch_write(query_productos, productos_rows)
    print(f"✓ {len(productos_rows)} productos cargados")

    # Clientes
    clientes_rows = (
        df_clientes[["id", "ciudad", "estado"]]
        .dropna(subset=["id"])
        .to_dict("records")
    )
    query_clientes = """
    UNWIND $rows AS row
    MERGE (c:Cliente {id: row.id})
    SET c.ciudad = row.ciudad,
        c.estado = row.estado
    """
    batch_write(query_clientes, clientes_rows)
    print(f"✓ {len(clientes_rows)} clientes cargados")

    # Relaciones Cliente -> Producto (INTERACTUA)
    interacciones_rows = (
        df_interacciones[["cliente_id", "producto_id", "rating"]]
        .fillna({"rating": 1})
        .to_dict("records")
    )
    query_interacciones = """
    UNWIND $rows AS row
    MATCH (c:Cliente {id: row.cliente_id})
    MATCH (p:Producto {id: row.producto_id})
    MERGE (c)-[r:INTERACTUA]->(p)
    SET r.rating = row.rating
    """
    batch_write(query_interacciones, interacciones_rows, batch_size=2000)
    print(f"✓ {len(interacciones_rows)} interacciones cargadas")

    # Relación Producto -> Vendedor (marketplace si aplica)
    if "id" in df_vendedores.columns:
        marketplace_id = df_vendedores["id"].iloc[0]
        query_marketplace = """
        MATCH (v:Vendedor {id: $seller_id})
        MATCH (p:Producto)
        MERGE (p)-[:VENDIDO_POR]->(v)
        """
        neo4j_conn.execute(query_marketplace, {"seller_id": marketplace_id})

elif DATASET_MODE == "olist":
    print("\nCargando datos Olist a Neo4j...")

    MAX_ORDERS = 5000  # set None para cargar todo

    df_orders_load = df_orders.copy()
    df_order_items_load = df_order_items.copy()
    df_clientes_load = df_clientes.copy()

    if MAX_ORDERS is not None:
        df_orders_load = df_orders_load.head(MAX_ORDERS)
        order_ids = set(df_orders_load["order_id"].tolist())
        df_order_items_load = df_order_items_load[df_order_items_load["order_id"].isin(order_ids)]
        df_clientes_load = df_clientes_load[df_clientes_load["id"].isin(df_orders_load["customer_id"])]

    # Vendedores
    vendedores_rows = (
        df_vendedores[["id", "ciudad", "estado"]]
        .dropna(subset=["id"])
        .to_dict("records")
    )
    query_vendedores = """
    UNWIND $rows AS row
    MERGE (v:Vendedor {id: row.id})
    SET v.ciudad = row.ciudad,
        v.estado = row.estado
    """
    batch_write(query_vendedores, vendedores_rows)
    print(f"✓ {len(vendedores_rows)} vendedores cargados")

    # Productos
    productos_rows = (
        df_productos[["product_id", "nombre", "categoria", "precio_promedio", "total_ordenes"]]
        .rename(columns={"product_id": "id"})
        .fillna({"precio_promedio": 0, "total_ordenes": 0})
        .to_dict("records")
    )
    query_productos = """
    UNWIND $rows AS row
    MERGE (p:Producto {id: row.id})
    SET p.nombre = row.nombre,
        p.categoria = row.categoria,
        p.precio_promedio = row.precio_promedio,
        p.total_ordenes = row.total_ordenes
    """
    batch_write(query_productos, productos_rows)
    print(f"✓ {len(productos_rows)} productos cargados")

    # Clientes
    clientes_rows = (
        df_clientes_load[["id", "ciudad", "estado"]]
        .dropna(subset=["id"])
        .to_dict("records")
    )
    query_clientes = """
    UNWIND $rows AS row
    MERGE (c:Cliente {id: row.id})
    SET c.ciudad = row.ciudad,
        c.estado = row.estado
    """
    batch_write(query_clientes, clientes_rows)
    print(f"✓ {len(clientes_rows)} clientes cargados")

    # Ordenes
    ordenes_rows = (
        df_orders_load[["order_id", "order_status", "order_purchase_timestamp", "customer_id"]]
        .rename(columns={"order_id": "id", "order_status": "status", "order_purchase_timestamp": "fecha_compra"})
        .to_dict("records")
    )
    query_ordenes = """
    UNWIND $rows AS row
    MERGE (o:Orden {id: row.id})
    SET o.status = row.status,
        o.fecha_compra = row.fecha_compra
    WITH o, row
    MATCH (c:Cliente {id: row.customer_id})
    MERGE (c)-[:REALIZO]->(o)
    """
    batch_write(query_ordenes, ordenes_rows)
    print(f"✓ {len(ordenes_rows)} ordenes cargadas")

    # Relaciones de items (Orden -> Producto y Producto -> Vendedor)
    items_rows = (
        df_order_items_load[["order_id", "order_item_id", "product_id", "seller_id", "price", "freight_value"]]
        .rename(columns={"order_id": "order_id", "product_id": "product_id", "seller_id": "seller_id"})
        .fillna({"price": 0, "freight_value": 0})
        .to_dict("records")
    )

    query_items = """
    UNWIND $rows AS row
    MATCH (o:Orden {id: row.order_id})
    MATCH (p:Producto {id: row.product_id})
    MERGE (o)-[r:CONTIENE {item_id: row.order_item_id}]->(p)
    SET r.precio = row.price,
        r.flete = row.freight_value
    WITH p, row
    MATCH (v:Vendedor {id: row.seller_id})
    MERGE (p)-[:VENDIDO_POR]->(v)
    """
    batch_write(query_items, items_rows, batch_size=2000)
    print(f"✓ {len(items_rows)} items cargados")

else:
    print("\nCargando dataset local a Neo4j...")

    # Crear nodos de Vendedores
    for _, vendedor in df_vendedores.iterrows():
        query = """
        CREATE (v:Vendedor {
            id: $id,
            nombre: $nombre,
            email: $email,
            ciudad: $ciudad,
            calificacion: $calificacion,
            productos_vendidos: $productos_vendidos,
            especialidad: $especialidad
        })
        """
        neo4j_conn.execute(query, vendedor.to_dict())

    # Crear nodos de Productos y relaciones
    for _, producto in df_productos.iterrows():
        query_producto = """
        CREATE (p:Producto {
            id: $id,
            nombre: $nombre,
            categoria: $categoria,
            precio: $precio,
            stock: $stock,
            ubicacion: $ubicacion,
            descripcion: $descripcion
        })
        """
        neo4j_conn.execute(
            query_producto,
            {
                "id": int(producto["id"]),
                "nombre": producto["nombre"],
                "categoria": producto["categoria"],
                "precio": float(producto["precio"]),
                "stock": int(producto["stock"]),
                "ubicacion": producto["ubicacion"],
                "descripcion": producto["descripcion"],
            },
        )

        query_relacion = """
        MATCH (p:Producto {id: $producto_id})
        MATCH (v:Vendedor {id: $vendedor_id})
        CREATE (p)-[:VENDIDO_POR]->(v)
        """
        neo4j_conn.execute(
            query_relacion,
            {"producto_id": int(producto["id"]), "vendedor_id": producto["vendedor"]},
        )

    print(f"✓ {len(df_productos)} productos creados con relaciones")

# Verificar
result = neo4j_conn.query("MATCH (n) RETURN count(n) as total")
print(f"\n✓ Total nodos en Neo4j: {result[0]['total']}")

Limpiando base de datos...

Cargando datos KaggleHub a Neo4j...
✓ 1 vendedores cargados
✓ 1474 productos cargados
✓ 1 clientes cargados
✓ 1474 interacciones cargadas

✓ Total nodos en Neo4j: 1476


## 6. Tabla de Funciones con Embeddings

In [7]:
# Importar tabla de funciones
from funciones_sistema import FUNCIONES_SISTEMA

# Generar embeddings para cada función
print("Generando embeddings para funciones del sistema...\n")
for funcion in FUNCIONES_SISTEMA:
    # Combinar descripción y ejemplos para embedding
    texto_completo = f"{funcion['nombre_funcion']}: {funcion['descripcion']}. Ejemplos: {', '.join(funcion['query_examples'][:3])}"
    funcion['embedding'] = embedder.encode(texto_completo)
    print(f"✓ {funcion['nombre_funcion']}")

# Crear DataFrame para visualización
# Tabla solicitada: id, nombre_funcion, descripcion, embedding, query_examples
# (embedding se muestra como longitud para mantener legible)
df_funciones = pd.DataFrame([{
    'id': f['id'],
    'nombre_funcion': f['nombre_funcion'],
    'descripcion': f['descripcion'],
    'embedding_dim': len(f['embedding']),
    'query_examples': "; ".join(f['query_examples'][:3])
} for f in FUNCIONES_SISTEMA])

print("\n=== TABLA DE FUNCIONES ===")
display(df_funciones)

Generando embeddings para funciones del sistema...

✓ buscar_producto
✓ buscar_vendedor
✓ verificar_stock
✓ comparar_precios
✓ crear_pedido
✓ consultar_categoria
✓ buscar_por_ubicacion
✓ obtener_recomendaciones

=== TABLA DE FUNCIONES ===


,id,nombre_funcion,descripcion,embedding_dim,query_examples
0,1,buscar_producto,"Busca productos en el inventario por nombre, c...",384,¿Cuanto cuesta el iPhone 14?; ¿Que laptops tie...
1,2,buscar_vendedor,"Encuentra vendedores por ciudad, especialidad ...",384,¿Quien vende smartphones en Lima?; Vendedores ...
2,3,verificar_stock,Verifica disponibilidad y cantidad de producto...,384,¿Cuantos iPhone 14 hay disponibles?; Stock de ...
3,4,comparar_precios,Compara precios entre productos similares o de...,384,¿Que smartphone es mas barato?; Comparar preci...
4,5,crear_pedido,Crea un nuevo pedido de compra. Retorna: pedid...,384,Quiero comprar un iPhone 14; Crear pedido de M...
5,6,consultar_categoria,Lista productos por categoria. Retorna: produc...,384,¿Que productos de Audio tienen?; Mostrar todos...
6,7,buscar_por_ubicacion,Filtra productos y vendedores por ciudad. Reto...,384,Productos disponibles en Lima; ¿Que venden en ...
7,8,obtener_recomendaciones,Sugiere productos relacionados o complementari...,384,¿Que me recomiendas con una laptop?; Productos...


## 7. Sistema de Routing (Selection Function)

## 7.1. Flujo hasta la selección de la función

En esta sección se visualiza el recorrido:
1. Query del usuario
2. Embedding de la consulta
3. Similitud con funciones
4. Función seleccionada

In [8]:
def seleccionar_funcion(query_usuario):
    """
    Selecciona la función más apropiada basada en similitud de embeddings
    con reglas simples de prioridad para consultas de precio o stock.
    """
    q = query_usuario.lower()

    price_keywords = ["precio", "cuesta", "vale", "costo", "cuanto"]
    stock_keywords = ["stock", "disponible", "disponibilidad", "hay"]

    if any(k in q for k in price_keywords):
        mejor = next((f for f in FUNCIONES_SISTEMA if f["nombre_funcion"] == "buscar_producto"), None)
        if mejor:
            return {"funcion": "buscar_producto", "similitud": 1.0, "id": mejor["id"], "descripcion": mejor["descripcion"]}, [
                {"funcion": "buscar_producto", "similitud": 1.0, "id": mejor["id"], "descripcion": mejor["descripcion"]}
            ]

    if any(k in q for k in stock_keywords):
        mejor = next((f for f in FUNCIONES_SISTEMA if f["nombre_funcion"] == "verificar_stock"), None)
        if mejor:
            return {"funcion": "verificar_stock", "similitud": 1.0, "id": mejor["id"], "descripcion": mejor["descripcion"]}, [
                {"funcion": "verificar_stock", "similitud": 1.0, "id": mejor["id"], "descripcion": mejor["descripcion"]}
            ]

    # Generar embedding de la query
    query_embedding = embedder.encode(query_usuario)

    # Calcular similitud con cada función
    similitudes = []
    for funcion in FUNCIONES_SISTEMA:
        sim = cosine_similarity([query_embedding], [funcion["embedding"]])[0][0]
        similitudes.append({
            "funcion": funcion["nombre_funcion"],
            "similitud": float(sim),
            "id": funcion["id"],
            "descripcion": funcion["descripcion"],
        })

    # Ordenar por similitud
    similitudes.sort(key=lambda x: x["similitud"], reverse=True)

    return similitudes[0], similitudes[:3]


# Test
test_queries = [
    "¿Cuanto cuesta el iPhone 14?",
    "¿Quien vende laptops en Lima?",
    "Quiero comprar audifonos",
]

print("=== TEST DE ROUTING ===\n")
for q in test_queries:
    mejor, top3 = seleccionar_funcion(q)
    print(f"Query: '{q}'")
    print(f"  → Función: {mejor['funcion']} (similitud: {mejor['similitud']:.3f})")
    print()

=== TEST DE ROUTING ===

Query: '¿Cuanto cuesta el iPhone 14?'
  → Función: buscar_producto (similitud: 1.000)

Query: '¿Quien vende laptops en Lima?'
  → Función: buscar_vendedor (similitud: 0.544)

Query: 'Quiero comprar audifonos'
  → Función: consultar_categoria (similitud: 0.445)



In [9]:
# Demostración del flujo hasta selección de función
consulta_demo = "¿Cuánto cuesta el iPhone 14?"

print("Paso 1: Query del usuario")
print(f"  {consulta_demo}\n")

print("Paso 2: Embedding de la consulta")
emb_demo = embedder.encode(consulta_demo)
print(f"  Dimensión del embedding: {len(emb_demo)}\n")

print("Paso 3: Similitud con funciones")
mejor, top3 = seleccionar_funcion(consulta_demo)
print("  Top 3 funciones:")
for f in top3:
    print(f"   - {f['funcion']}: {f['similitud']:.3f}")

print("\nPaso 4: Función seleccionada")
print(f"  {mejor['funcion']}")

Paso 1: Query del usuario
  ¿Cuánto cuesta el iPhone 14?

Paso 2: Embedding de la consulta
  Dimensión del embedding: 384

Paso 3: Similitud con funciones
  Top 3 funciones:
   - buscar_producto: 1.000

Paso 4: Función seleccionada
  buscar_producto


## 8. Implementación de Funciones (Planner)

In [10]:
def buscar_producto(query, params=None):
    """Busca productos en Neo4j"""
    cypher = """
    MATCH (p:Producto)
    OPTIONAL MATCH (p)-[:VENDIDO_POR]->(v:Vendedor)
    RETURN p.nombre as producto,
           p.categoria as categoria,
           coalesce(p.precio, p.precio_promedio) as precio,
           coalesce(p.stock, p.total_ordenes) as stock,
           v.id as vendedor,
           v.ciudad as ciudad
    ORDER BY coalesce(p.precio, p.precio_promedio)
    LIMIT 10
    """
    return neo4j_conn.query(cypher)


def buscar_vendedor(query, params=None):
    """Busca vendedores en Neo4j"""
    cypher = """
    MATCH (v:Vendedor)
    RETURN v.id as vendedor,
           v.ciudad as ciudad,
           v.estado as estado,
           v.calificacion as calificacion,
           v.especialidad as especialidad
    ORDER BY v.calificacion DESC
    """
    return neo4j_conn.query(cypher)


def verificar_stock(query, params=None):
    """Verifica disponibilidad de productos"""
    cypher = """
    MATCH (p:Producto)
    RETURN p.nombre as producto,
           coalesce(p.stock, p.total_ordenes) as disponible,
           coalesce(p.precio, p.precio_promedio) as precio,
           p.categoria as categoria
    ORDER BY disponible DESC
    LIMIT 20
    """
    return neo4j_conn.query(cypher)


def comparar_precios(query, params=None):
    """Compara precios entre productos"""
    cypher = """
    MATCH (p:Producto)
    RETURN p.nombre as producto,
           p.categoria as categoria,
           coalesce(p.precio, p.precio_promedio) as precio
    ORDER BY p.categoria, coalesce(p.precio, p.precio_promedio)
    LIMIT 50
    """
    return neo4j_conn.query(cypher)


def consultar_categoria(query, params=None):
    """Lista productos por categoría"""
    cypher = """
    MATCH (p:Producto)
    RETURN p.categoria as categoria,
           count(p) as cantidad_productos,
           min(coalesce(p.precio, p.precio_promedio)) as precio_minimo,
           max(coalesce(p.precio, p.precio_promedio)) as precio_maximo
    ORDER BY cantidad_productos DESC
    """
    return neo4j_conn.query(cypher)


def buscar_por_ubicacion(query, params=None):
    """Filtra por ubicación de vendedores"""
    cypher = """
    MATCH (v:Vendedor)
    RETURN v.ciudad as ciudad,
           count(v) as vendedores
    ORDER BY vendedores DESC
    """
    return neo4j_conn.query(cypher)


# Mapeo de funciones
FUNCION_EXECUTOR = {
    "buscar_producto": buscar_producto,
    "buscar_vendedor": buscar_vendedor,
    "verificar_stock": verificar_stock,
    "comparar_precios": comparar_precios,
    "consultar_categoria": consultar_categoria,
    "buscar_por_ubicacion": buscar_por_ubicacion,
}

print("✓ Funciones del planner implementadas")

✓ Funciones del planner implementadas


## 9. Integración con LLM (Response Generation)

In [11]:
def _seleccionar_modelo_gemini():
    """Selecciona un modelo disponible que soporte generateContent."""
    try:
        modelos = list(genai.list_models())
        candidatos = [m for m in modelos if "generateContent" in getattr(m, "supported_generation_methods", [])]
        # Preferencias comunes
        preferidos = [
            "models/gemini-1.5-flash",
            "models/gemini-1.5-pro",
            "models/gemini-pro",
        ]
        for nombre in preferidos:
            for m in candidatos:
                if m.name == nombre:
                    return m.name
        return candidatos[0].name if candidatos else None
    except Exception:
        return None


def generar_respuesta_llm(query_usuario, datos_neo4j, funcion_utilizada):
    """
    Usa Google Gemini para convertir datos estructurados en respuesta natural
    """
    if not GOOGLE_API_KEY:
        # Fallback: respuesta simple sin LLM
        return f"Función ejecutada: {funcion_utilizada}\n\nResultados:\n{json.dumps(datos_neo4j[:3], indent=2, ensure_ascii=False)}"

    if not datos_neo4j:
        return "No se encontraron resultados en la base de datos."

    # Preparar prompt para Gemini
    prompt = f"""Eres un asistente de una tienda online. Un cliente preguntó:

"{query_usuario}"

Se ejecutó la función '{funcion_utilizada}' y se obtuvieron estos datos de la base de datos:

{json.dumps(datos_neo4j[:5], indent=2, ensure_ascii=False)}

Genera una respuesta natural, amigable y concisa en español. Si hay productos, menciona nombres, precios y vendedores. Si son vendedores, menciona ciudades y especialidades."""

    try:
        modelo = _seleccionar_modelo_gemini()
        if not modelo:
            return "No se encontró un modelo Gemini disponible para generateContent."
        model = genai.GenerativeModel(modelo)
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"Error al generar respuesta LLM: {e}\n\nDatos: {json.dumps(datos_neo4j[:3], indent=2, ensure_ascii=False)}"


print("✓ Generador de respuestas LLM (Google Gemini) configurado")

✓ Generador de respuestas LLM (Google Gemini) configurado


## 10. Pipeline Completo

In [12]:
def ejecutar_pipeline(query_usuario, verbose=True):
    """
    Pipeline completo: Query → Selection → Planner → LLM Response
    """
    if verbose:
        print("\n" + "="*70)
        print("=== PIPELINE TRANSACCIONAL ===")
        print("="*70)
        print(f"\nQuery: '{query_usuario}'\n")
    
    # 1. SELECTION FUNCTION (Routing)
    if verbose:
        print("[1/3] SELECTION FUNCTION - Routing de función")
    
    mejor_funcion, top3 = seleccionar_funcion(query_usuario)
    
    if verbose:
        print(f"  → Función seleccionada: {mejor_funcion['funcion']}")
        print(f"  → Similitud: {mejor_funcion['similitud']:.3f}")
        print(f"  → Top 3: {', '.join([f['funcion'] for f in top3])}\n")
    
    # 2. PLANNER (Ejecutar función en Neo4j)
    if verbose:
        print("[2/3] PLANNER - Ejecución en Neo4j")
    
    nombre_funcion = mejor_funcion['funcion']
    
    if nombre_funcion in FUNCION_EXECUTOR:
        funcion = FUNCION_EXECUTOR[nombre_funcion]
        resultados_neo4j = funcion(query_usuario)
        
        if verbose:
            print(f"  → Resultados obtenidos: {len(resultados_neo4j)} registros")
            if resultados_neo4j:
                print(f"  → Campos: {', '.join(resultados_neo4j[0].keys())}\n")
    else:
        resultados_neo4j = []
        if verbose:
            print(f"  → Función no implementada aún\n")
    
    # 3. LLM RESPONSE (Generar respuesta en lenguaje natural)
    if verbose:
        print("[3/3] LLM RESPONSE - Generación de respuesta")
    
    respuesta_final = generar_respuesta_llm(query_usuario, resultados_neo4j, nombre_funcion)
    
    if verbose:
        print("\n" + "="*70)
        print("=== RESPUESTA FINAL ===")
        print("="*70)
    
    print(f"\n{respuesta_final}\n")
    
    return {
        'query': query_usuario,
        'funcion': nombre_funcion,
        'similitud': mejor_funcion['similitud'],
        'resultados_neo4j': resultados_neo4j,
        'respuesta': respuesta_final
    }

print("✓ Pipeline completo configurado")

✓ Pipeline completo configurado


## 11. Ejecución de Consultas

In [13]:
# Consulta interactiva
try:
    query_usuario = input("Ingrese su consulta: ").strip()
except:
    query_usuario = ""

if not query_usuario:
    query_usuario = "¿Cuanto cuesta el iPhone 14?"

resultado = ejecutar_pipeline(query_usuario)


=== PIPELINE TRANSACCIONAL ===

Query: '¿Cuanto cuesta el iPhone 14?'

[1/3] SELECTION FUNCTION - Routing de función
  → Función seleccionada: buscar_producto
  → Similitud: 1.000
  → Top 3: buscar_producto

[2/3] PLANNER - Ejecución en Neo4j


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: precio_promedio)} {position: line: 6, column: 33, offset: 178} for query: '\n    MATCH (p:Producto)\n    OPTIONAL MATCH (p)-[:VENDIDO_POR]->(v:Vendedor)\n    RETURN p.nombre as producto,\n           p.categoria as categoria,\n           coalesce(p.precio, p.precio_promedio) as precio,\n           coalesce(p.stock, p.total_ordenes) as stock,\n           v.id as vendedor,\n           v.ciudad as ciudad\n    ORDER BY coalesce(p.precio, p.precio_promedio)\n    LIMIT 10\n    '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotif

  → Resultados obtenidos: 10 registros
  → Campos: producto, categoria, precio, stock, vendedor, ciudad

[3/3] LLM RESPONSE - Generación de respuesta

=== RESPUESTA FINAL ===

¡Hola! Gracias por tu pregunta.

He buscado el iPhone 14 en nuestra base de datos, pero no he encontrado ningún producto con ese nombre específico en este momento.

La búsqueda arrojó algunos productos genéricos (con ID numérico) que tienen un precio de 100 y son vendidos por "marketplace", pero no corresponden a un iPhone 14. Es posible que el producto no esté disponible actualmente o que su nombre sea ligeramente diferente.

¿Te gustaría que intente buscar otra cosa o te interese algún otro modelo de iPhone?



## 12. Ejemplos de Consultas

In [14]:
consultas_ejemplo = [
    "¿Cuanto cuesta el iPhone 14?",
    "¿Quien vende laptops en Lima?",
    "¿Hay stock de audifonos Sony?",
    "¿Que smartphone es mas barato?",
    "¿Que productos tienen en la categoria Audio?"
]

print("\n" + "="*70)
print("=== PROCESAMIENTO BATCH ===")
print("="*70)

resultados_batch = []
for consulta in consultas_ejemplo:
    print(f"\n{'='*70}")
    resultado = ejecutar_pipeline(consulta, verbose=False)
    resultados_batch.append(resultado)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: precio_promedio)} {position: line: 6, column: 33, offset: 178} for query: '\n    MATCH (p:Producto)\n    OPTIONAL MATCH (p)-[:VENDIDO_POR]->(v:Vendedor)\n    RETURN p.nombre as producto,\n           p.categoria as categoria,\n           coalesce(p.precio, p.precio_promedio) as precio,\n           coalesce(p.stock, p.total_ordenes) as stock,\n           v.id as vendedor,\n           v.ciudad as ciudad\n    ORDER BY coalesce(p.precio, p.precio_promedio)\n    LIMIT 10\n    '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotif


=== PROCESAMIENTO BATCH ===


¡Hola! He buscado el iPhone 14 para ti.

Según nuestros resultados, no hemos encontrado un producto listado directamente con el nombre "iPhone 14". Sin embargo, se encontraron varios artículos que tienen un precio de 100 cada uno y son vendidos por nuestro marketplace.

¿Te gustaría que investigáramos más a fondo o necesitas buscar otra cosa?




Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: total_ordenes)} {position: line: 4, column: 32, offset: 88} for query: '\n    MATCH (p:Producto)\n    RETURN p.nombre as producto,\n           coalesce(p.stock, p.total_ordenes) as disponible,\n           coalesce(p.precio, p.precio_promedio) as precio,\n           p.categoria as categoria\n    ORDER BY disponible DESC\n    LIMIT 20\n    '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {descri


¡Hola! He buscado vendedores de laptops para ti.

Encontré un vendedor general llamado **"marketplace"**. Según la información que tengo, no se especifica una ciudad o especialidad en particular (aparece como 'desconocido'). Un 'marketplace' es una plataforma donde varios vendedores ofrecen sus productos.

Dado que no tengo detalles de ubicaciones específicas en Lima para este vendedor, ¿te gustaría que busque directamente laptops que estén disponibles y que puedan ser enviadas a Lima, o quizás tienes alguna marca o característica específica en mente?




Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: precio_promedio)} {position: line: 5, column: 33, offset: 126} for query: '\n    MATCH (p:Producto)\n    RETURN p.nombre as producto,\n           p.categoria as categoria,\n           coalesce(p.precio, p.precio_promedio) as precio\n    ORDER BY p.categoria, coalesce(p.precio, p.precio_promedio)\n    LIMIT 50\n    '



¡Hola! Gracias por tu consulta.

He verificado la disponibilidad de audífonos Sony y, según la información actual, no me aparece ninguna unidad disponible en este momento. Parece que no hay stock para esos modelos específicos o la información no está actualizada.

Si gustas, puedo ayudarte a buscar otras opciones de audífonos que sí tengamos en stock de otras marcas o modelos similares. ¡Avísame si te interesa!




Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: precio_promedio)} {position: line: 5, column: 37, offset: 140} for query: '\n    MATCH (p:Producto)\n    RETURN p.categoria as categoria,\n           count(p) as cantidad_productos,\n           min(coalesce(p.precio, p.precio_promedio)) as precio_minimo,\n           max(coalesce(p.precio, p.precio_promedio)) as precio_maximo\n    ORDER BY cantidad_productos DESC\n    '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key i


¡Hola! He buscado los smartphones más económicos para ti.

He encontrado que varios productos comparten el precio más bajo, todos a **100**.

Lamento informarte que, con la información actual de la base de datos, no tengo los nombres específicos de los modelos de smartphone ni los detalles de los vendedores para poder darte más detalles.

¿Hay algo más en lo que pueda ayudarte o alguna característica que te interese para filtrar la búsqueda?



¡Hola! Gracias por tu interés en la categoría de **Audio**.

Parece que en este momento no tengo detalles específicos sobre productos de **Audio** directamente con esa búsqueda. Los datos que obtuve son más bien generales sobre nuestra tienda, indicando que tenemos **1474 productos en total**, con precios que van desde los **$100 hasta los $19,000**.

Me disculpo por no poder darte la información exacta de Audio ahora mismo. Podría ser un pequeño error, o tal vez esa categoría específica no está cargada con detalles por el momento.

Te sugiero 

## 13. Visualización de Resultados

In [15]:
# Resumen de funciones utilizadas
df_resumen = pd.DataFrame([{
    'Consulta': r['query'][:50] + '...',
    'Función': r['funcion'],
    'Similitud': f"{r['similitud']:.3f}",
    'Resultados': len(r['resultados_neo4j'])
} for r in resultados_batch])

print("\n=== RESUMEN DE EJECUCIONES ===")
display(df_resumen)


=== RESUMEN DE EJECUCIONES ===


,Consulta,Función,Similitud,Resultados
0,¿Cuanto cuesta el iPhone 14?...,buscar_producto,1.000,10
1,¿Quien vende laptops en Lima?...,buscar_vendedor,0.544,1
2,¿Hay stock de audifonos Sony?...,verificar_stock,1.000,20
3,¿Que smartphone es mas barato?...,comparar_precios,0.568,50
4,¿Que productos tienen en la categoria Audio?...,consultar_categoria,0.767,1


## 14. Cleanup

In [42]:
# Cerrar conexión Neo4j
try:
    neo4j_conn.close()
    print("✓ Conexión Neo4j cerrada")
except:
    pass

✓ Conexión Neo4j cerrada


## Verificación del flujo del agente (LangChain + LangGraph)
Este bloque muestra logs secuenciales, embeddings explícitos y un plan de ejecución con LangGraph.

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langgraph.graph import StateGraph, END
from sklearn.metrics.pairwise import cosine_similarity

# Input
query = "quiero 2 iPhone 14 Pro"
print(f"input: {query}")

# Embeddings (modelo explícito)
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
emb = HuggingFaceEmbeddings(model_name=model_name)
print(f"embedding_model: {model_name}")
q_emb = emb.embed_query(query)

# Function selection (similitud)
funciones = {
    "buscar_producto": "Busca productos por nombre o categoria",
    "verificar_stock": "Consulta disponibilidad de productos",
    "buscar_vendedor": "Busca vendedores",
}
func_embs = {k: emb.embed_query(v) for k, v in funciones.items()}
scored = sorted([(k, float(cosine_similarity([q_emb], [v])[0][0])) for k, v in func_embs.items()], key=lambda x: x[1], reverse=True)
selected = scored[0][0]
print(f"seleccion_funcion: {selected}")

# Plan con LangGraph (explícito y secuencial)
plan = [selected]
print(f"plan: {plan}")

state = {"query": query, "logs": []}

graph = StateGraph(dict)

def step_buscar_producto(s):
    s["logs"].append("step: buscar_producto")
    print("step: buscar_producto")
    return s

def step_verificar_stock(s):
    s["logs"].append("step: verificar_stock")
    print("step: verificar_stock")
    return s

def step_buscar_vendedor(s):
    s["logs"].append("step: buscar_vendedor")
    print("step: buscar_vendedor")
    return s

node_map = {
    "buscar_producto": step_buscar_producto,
    "verificar_stock": step_verificar_stock,
    "buscar_vendedor": step_buscar_vendedor,
}

prev = None
for i, step in enumerate(plan):
    name = f"step_{i}_{step}"
    graph.add_node(name, node_map.get(step, step_buscar_producto))
    if prev is None:
        graph.set_entry_point(name)
    else:
        graph.add_edge(prev, name)
    prev = name

graph.add_edge(prev, END)
app = graph.compile()
app.invoke(state)

print("output: respuesta natural generada")

input: quiero 2 iPhone 14 Pro


C:\Users\mlata\AppData\Local\Temp\ipykernel_21892\4101831580.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  emb = HuggingFaceEmbeddings(model_name=model_name)


embedding_model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
seleccion_funcion: verificar_stock
plan: ['verificar_stock']
step: verificar_stock
output: respuesta natural generada


: 